In [19]:
import os
os.getcwd()

'/home/jovyan/ArditArifi'

In [20]:
source_path="/home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6"
output_path="/home/jovyan/student-storages/GROUP3/ArditArifi/output"

! mkdir -p output_path

Analysis

In [29]:
import glob
# ANALYSIS
import xarray as xr
import pandas as p
import numpy as np
# PLOTTING
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.colors as colors
# USER
import area_weight as aw


In [26]:

# REGIONS
region_label = ["ARC", "ANT"]
arctic_lat = [60, 90]
antarctic_lat = [-90, -60]

# SEASONS
seasons_label = ["DJF", "MAM", "JJA", "SON"]
seasons = {
    "DJF": [12, 1, 2],   # Dec, Jan, Feb
    "MAM": [3, 4, 5],    # Mar, Apr, May
    "JJA": [6, 7, 8],    # Jun, Jul, Aug
    "SON": [9, 10, 11],  # Sep, Oct, Nov
}

# Create combined region-season labels
region_season_label = [f"{region} {season}" for region in region_label for season in seasons_label]

# EXPERIMENTS
experiments = ["2xss", "2xdust", "2xfire", "2xDMS"]

# MODELS
models = ["NorESM2-LM"] # ["IPSL-CM6A-LR-INCA", "UKESM1-0-LL", "NorESM2-LM"]

# PATTERN FUNCTION
patter = lambda model, var, exp: f"{source_path}/{var}*{model}*-{exp}*.nc"

In [28]:
# CONSTRUCT TABLE
mean_values = np.full(
    shape=(len(region_season_label), len(experiments), len(models)),
    fill_value=np.nan
)
std_values = np.full(
    shape=(len(region_season_label), len(experiments), len(models)),
    fill_value=np.nan
)


# LOAD DATA
model = ""
var = "rtmt"

# Calculate total steps for progress
total_steps = len(models) * len(experiments) * len(seasons_label)
current_step = 0

# MAIN LOOP
for idx_model, model in enumerate(models):
    try:
        # Open control dataset for this model
        print(patter(model, var, "control"))
        ds_ctl = xr.open_mfdataset(patter(model, var, "control"), combine="by_coords")

    except Exception as e:
        print(f"Error opening control files {patter(model, var, 'control')}: {e}")
        # Skip all seasons for this model if control is missing
        current_step += len(seasons_label) * len(experiments)
        continue
        
    for idx_exp, exp in enumerate(experiments):
        # Try opening the experiment dataset
        try:
            ds_exp = xr.open_mfdataset(patter(model, var, exp), combine="by_coords" )
        except Exception as e:
            print(f"Error opening experiment files {patter(model, var, exp)}: {e}")
            current_step += len(seasons_label)
            continue

        # Compute bias as experiment minus control
        ds_bias = ds_exp[var] - ds_ctl[var]

        for idx_season, season in enumerate(seasons_label):
            # Increment the progress
            current_step += 1
            # Compute and display percentage
            pct = (current_step / total_steps) * 100
            print(f"Progress: {current_step}/{total_steps} -> {pct:.2f}%")
            
            # Select months for this season
            ds_season = ds_bias.sel(time=ds_bias.time.dt.month.isin(seasons[season]))

            # ---------------------
            # ARCTIC
            # ---------------------
            ds_season_arctic = ds_season[var].sel(
                lat=slice(arctic_lat[0], arctic_lat[1]),
            )
            lat_2d = ds_season_arctic["lat"].broadcast_like(ds_season_arctic)
            weights_arctic_2d = np.cos(np.deg2rad(lat_2d))
            
            weights_arctic = np.cos(np.deg2rad(weights_arctic_2d))

            # Weighted mean over lat/lon => time series
            ds_season_arctic_area_ts = ds_season_arctic.weighted(weights_arctic).mean(dim=("lat", "lon"))

            # Multi-year average (time)
            arc_mean = ds_season_arctic_area_ts.mean("time").values
            mean_values[idx_season, idx_exp, idx_model] = arc_mean

            arc_std = ds_season_arctic_area_ts.std("time").values
            std_values[idx_season, idx_exp, idx_model] = arc_std
            
            # ---------------------
            # ANTARCTIC
            # ---------------------
            ds_season_antarctic = ds_season[var].sel(
                lat=slice(antarctic_lat[0], antarctic_lat[1]),
            )
            weights_antarctic = np.cos(np.deg2rad(ds_season_antarctic.lat))

            ds_season_antarctic_area_ts = ds_season_antarctic.weighted(weights_antarctic).mean(dim=("lat","lon"))

            # Multi-year average (time)
            ant_mean = ds_season_antarctic_area_ts.mean("time").values
            mean_values[idx_season + 4, idx_exp, idx_model] = ant_mean

            print(ant_mean)
            
            ant_std = ds_season_antarctic_area_ts.std("time").values
            std_values[idx_season + 4, idx_exp, idx_model] = ant_std
            
# Finally, display the populated array
print("Shape of mean_values:", mean_values.shape)
print(mean_values)

/home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6/rtmt*NorESM2-LM*-control*.nc
Progress: 1/16 -> 6.25%


KeyError: 'rtmt'

In [25]:
import pandas as pd

bias = mean_values
for idx_exp in range(len(experiments)):
    # This will skip the control experiment at index 0
    exp_name = experiments[idx_exp]
    print("Experiment:", exp_name)
    
    # bias[:, idx_exp, :].shape -> (len(region_season_label), len(models))
    # bias[:, idx_exp, :].transpose().shape -> (len(models), len(region_season_label))
    # We want models as rows and region_season_label as columns
    df = pd.DataFrame(
        data=bias[:, idx_exp, :].transpose(),
        index=models,
        columns=region_season_label
    )

    print(df)
    print()  # Blank line for readability

Experiment: 2xss
                    ARC DJF   ARC MAM   ARC JJA   ARC SON   ANT DJF   ANT MAM  \
IPSL-CM6A-LR-INCA  0.360479 -0.693598 -0.514919 -0.079786 -0.639208 -0.149330   
UKESM1-0-LL       -0.947611  0.379108 -0.299512 -0.139673 -0.696725 -0.341245   
NorESM2-LM              NaN       NaN       NaN       NaN       NaN       NaN   

                    ANT JJA   ANT SON  
IPSL-CM6A-LR-INCA  0.025352 -0.049143  
UKESM1-0-LL        0.166987 -0.584054  
NorESM2-LM              NaN       NaN  

Experiment: 2xdust
                    ARC DJF   ARC MAM   ARC JJA   ARC SON   ANT DJF   ANT MAM  \
IPSL-CM6A-LR-INCA  0.751637 -0.676892 -0.080858  0.543402 -0.087152  0.115628   
UKESM1-0-LL       -0.304494  0.358536 -0.161269  0.031659  0.007193 -0.104539   
NorESM2-LM              NaN       NaN       NaN       NaN       NaN       NaN   

                    ANT JJA   ANT SON  
IPSL-CM6A-LR-INCA -0.007568  0.234224  
UKESM1-0-LL        0.150107 -0.024465  
NorESM2-LM              NaN      

In [23]:
import pandas as pd
import numpy as np

# Suppose the below variables are already defined
# bias : shape (len(region_season_label), len(experiments)-1, len(models))
# region_season_label : list of region/season names
# experiments : list of experiments, with experiment[0] = "control"
# models : list of model names

all_data = []  # Will hold the melted DataFrame for each experiment

for idx_exp in range(len(experiments) - 1):
    exp_name = experiments[idx_exp + 1]
    print("Experiment:", exp_name)

    # bias[:, idx_exp, :].shape -> (n_region_season, n_models)
    # transpose -> (n_models, n_region_season)
    df = pd.DataFrame(
        data=bias[:, idx_exp, :].transpose(),
        index=models,
        columns=region_season_label
    )

    # Reshape df to "long" format with columns [Model, region_season_label, value]
    df_long = df.reset_index().melt(
        id_vars="index",                 # "index" column is the model
        var_name="RegionSeason",         # name for the melted columns
        value_name="Bias"               # name for the melted values
    )

    # Rename "index" -> "Model"
    df_long.rename(columns={"index": "Model"}, inplace=True)

    # Add a column for Experiment
    df_long["Experiment"] = exp_name

    # Append to the list
    all_data.append(df_long)

# Concatenate all experiments into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Optionally reorder columns (Experiment, Model, RegionSeason, Bias)
final_df = final_df[["Experiment", "Model", "RegionSeason", "Bias"]]

# Save to CSV
final_df.to_csv("rtmt_seq.csv", index=False)

print("CSV file saved: all_bias_results.csv")


Experiment: 2xss
Experiment: 2xdust
Experiment: 2xfire
Experiment: 2xDMS
CSV file saved: all_bias_results.csv
